## Kubernetes Configuration

This notebook can be used to configure your Kubernetes Cluster.


In [1]:
#This will import some required libraries.
import sys 
import ruamel.yaml 
#This is your configuration file. 
general_yaml='../../config/config.yaml'
with open(general_yaml, 'r') as yaml:
    cf=ruamel.yaml.round_trip_load(yaml, preserve_quotes=True)

jupyterhub_yaml='../../config/jupyterhub/config.yaml'
with open(jupyterhub_yaml, 'r') as j_yaml:
    cf_j=ruamel.yaml.round_trip_load(j_yaml, preserve_quotes=True)

    #This will allow us to import some useful utilities. 
if cf['docker']:
    cf_j['path']=cf['docker_path']
else:
    cf_j['path']=cf['local_path']

sys.path.append(cf_j['path']+"/lib/kuberutils")   
import importlib
import kuberutils as ku
importlib.reload(ku)
#This will load common commands for your cluster
cf_j=ku.jupyterhub_commands(cf_j)
print(ruamel.yaml.dump(cf_j, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))



rebuild_config: false
url: lab.analyticsdojo.com
ssl: false
email: jkuruzovich@gmail.com    #This is used for letsencrypt
namespace: dojo      #Generally the namespace and releasename should be teh same.
releasename: dojo
fixed_ip: 35.185.85.199
set_fixed_ip: true
jupyterhub_chart: /Users/jasonkuruzovich/githubdesktop/admin-tools/helm/jupyterhub-v0.4.1.tgz
#  jupyterhub_chart: "https://github.com/jupyterhub/helm-chart/releases/download/v0.4/jupyterhub-v0.4.tgz"
jupyterhub_helm_repo: https://jupyterhub.github.io/helm-chart/
jupyterhub_version: v0.4
jupyterhub_config_init: https://raw.githubusercontent.com/jupyterhub/helm-chart/master/jupyterhub/values.yaml
github_auth:
  type: github
  github:
    clientId: "e106e1c6e831461a94c2"
    clientSecret: "e4427fac76d117d1201b509d793f1eccde2c7c83"
    callbackUrl: "http://lab.analyticsdojo.com/hub/oauth_callback"
google_auth:
  type: google
  google:
    clientId: "yourlongclientidstring.apps.googleusercontent.com"
    clientSecret: "adifferent

#This loads the core configuration file.  
kube_yaml='../../config/jupyterkubernetes.yaml'
import ruamel.yaml 
with open(kube_yaml, 'r') as yaml:
    cf=ruamel.yaml.round_trip_load(yaml, preserve_quotes=True)
#print(ruamel.yaml.round_trip_dump(cf))

#This just creates some additional varaibles. 
namespace= cf['jupyterhub']['namespace']
releasename= cf['jupyterhub']['releasename']
jupyterhub_config_init=cf['jupyterhub']['jupyterhub_config_init']
jupyterhub_chart=cf['jupyterhub']['jupyterhub_chart']
path="../../config/"+namespace
jupyterhub_config=path+"/values.yaml"

### Kubernetes Dashboard
We can also get some general information about the Kubernetes cluster from the builtin dashboard system. 

*Cluster Info*:  This will give the public URL of the dashboard.

Unless you specified it when creating your cluster, the default user is admin.  To To get the password you will need call the `!gcloud container clusters describe $cluster_name` 

In [2]:
#You can use this to show the Kubernetes Dashboard.
!kubectl cluster-info

Kubernetes master is running at https://35.196.99.23
GLBCDefaultBackend is running at https://35.196.99.23/api/v1/namespaces/kube-system/services/default-http-backend/proxy
Heapster is running at https://35.196.99.23/api/v1/namespaces/kube-system/services/heapster/proxy
KubeDNS is running at https://35.196.99.23/api/v1/namespaces/kube-system/services/kube-dns/proxy
kubernetes-dashboard is running at https://35.196.99.23/api/v1/namespaces/kube-system/services/kubernetes-dashboard/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [3]:
#This has the cluster username (admin) and password.
result=!gcloud container clusters describe $cluster_name
password= [x for x in result if "password:" in x]
print (password)

[]


### Kubernetes Nodes
Throughout this we have been connecting to the google cloud platform through `gcloud`.  We are now going to shift to the more general `kubectl`. You can see the wide variety of commands with the `kubectl --help` command.  

*Nodes*: For Kubernetes, [nodes](https://kubernetes.io/docs/concepts/architecture/nodes/) are worker machines or servers. By issuing the `get nodes` command, we can list our servers.  This command will work whether you are using Kubernetes on Google, Amazon, or Microsoft's Azure Platform. 

`kubectl get nodes`


In [4]:
#This should show the nodes
!kubectl get nodes

NAME                                         STATUS    AGE       VERSION
gke-kuberlytics-default-pool-3af057b0-rx6q   Ready     1m        v1.6.9


### Kubernetes Pods
"A (as in a pod of whales or pea pod) is a group of one or more containers (such as Docker containers), the shared storage for those containers, and options about how to run the containers." - [Kubernetes Documentation](https://kubernetes.io/docs/concepts/workloads/pods/pod/#what-is-a-pod)

Because the pods are specifically used do designate "one or more" containers, you can think of a pod as a single applications. Sometime that application can consist of a single container.  Sometimes it can consist of multiple containers. At times you may want to run all containers from an application together.

There are also *system* oriented pods on Kubernetes.  To see all pods, we issue the command:

`kubectl get pods --all-namespaces`

You should see the DNS, the dashboard, etc. 

In [7]:
#This should just initially show the pods associated with Kubernetes system. 
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                                    READY     STATUS    RESTARTS   AGE
dojo          hub-deployment-3772845711-3b01j                         1/1       Running   0          13m
dojo          proxy-deployment-51742714-cd54x                         1/1       Running   0          13m
dojo          pull--57e832b-1507077133-r4jv7                          1/1       Running   0          2m
dojo          pull-all-nodes-1507077128-dojo-24-gsmsb                 1/1       Running   0          2m
kube-system   fluentd-gcp-v2.0-p4h8t                                  2/2       Running   0          9m
kube-system   heapster-v1.3.0-2959597048-1n3vc                        2/2       Running   0          13m
kube-system   kube-dns-323615064-lx2xg                                3/3       Running   0          13m
kube-system   kube-dns-autoscaler-2667913178-s30jx                    1/1       Running   0          13m
kube-system   kube-proxy-gke-kuberlytics-default-pool-3af0

### Bash into a Container.


In [7]:
!echo kubectl --namespace=$namespace exec -it hub-deployment-1769082537-1zs0h  -- /bin/bash

kubectl --namespace= exec -it hub-deployment-1769082537-1zs0h -- /bin/bash


In [ ]:
!kubectl --namespace= exec -it hub-deployment-1769082537-1zs0h -- /bin/bash

### Kubernetes Services

"A Kubernetes Service is an abstraction which defines a logical set of Pods and a policy by which to access them - sometimes called a micro-service." 
- [Kubernetes Documentation](https://kubernetes.io/docs/concepts/services-networking/service/)

Pods are temporary, and can launch or die.  Services help applications keep track of a pod's location

In [ ]:
#This will show across all namespaces
!!kubectl get services --all-namespaces

In [ ]:
#This will get the external IP address
!kubectl --namespace=$namespace get svc proxy-public

In [ ]:
#This will show all services
!kubectl get services --all-namespaces

In [ ]:
#This could potentially enable autoscaling
!gcloud alpha container clusters update $cluster_name --enable-autoscaling --min-nodes=$num_nodes --max-nodes=$max_nodes --zone=$zone --node-pool=default-pool

In [ ]:
#This will resize a container to a different number of nodes.
!gcloud container clusters resize $cluster_name --size 1 --quiet


In [ ]:
#This will change the singleuser container
with open(config_jupyter, 'r') as yaml:
    data=ruamel.yaml.round_trip_load(yaml)
data['singleuser']['image']['name']="berkeleydsep/singleuser-data8"
data['singleuser']['image']['tag']="v0.1"
ruamel.yaml.round_trip_dump(data, open(config_jupyter, 'w'))

In [ ]:
#Updating the release to included 
!helm upgrade $releasename $jupyterhubchart -f $config_jupyter

In [ ]:
#This lists the pods. 
!kubectl --namespace=$namespace get pod